Introduction to Microsoft Agent Framework

topics of the SK 2.1:
1. One simple agent - done
2. Dynamic agent instructions with template parameters (tbd)
3. Multi-Turn Conversations - done
4. EX1 - Simple Question-Answering Agent - too simple?
5. Length of chat history in check - done
6. Function calling and plugins - done
7. Plugin example for agent - done
8. Function invocation deep-dive - removed
9. function calling modes - tbd
10. EX2 - create your own plugin - changing to middleware
11. agent that generates creative content and how streaming works - tbd
12. static error handling - tbd


the potential topics for 1.1 AF (single agent):
1. simple agent and agent run options (and streaming)
2. TBD [responses and messages](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/running-agents?pivots=programming-language-python)
    [ChatMessage object example](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/run-agent?pivots=programming-language-python)

3. [Multi-Turn Conversations and Threading](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python)
    Added an example for [this](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/multi-turn-conversation?pivots=programming-language-python)

4. TBD - introduce [AgentThread Storage and Custom Message Stores](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/multi-turn-conversation?pivots=programming-language-python) and [persisting agent conversations](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/persisted-conversation?pivots=programming-language-python) or [ChatMessage Store](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/third-party-chat-history-storage?pivots=programming-language-python) to align with SK 2.1 "Keeping the length of the chat history in check"


5. Agent function tools - https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-tools?pivots=programming-language-python
    Exercise to create an agent with function tools [based on this](https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/function-tools?pivots=programming-language-python)


6. Middleware - https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-middleware?pivots=programming-language-python
    6.1 potential exercise to add logging functions https://learn.microsoft.com/en-us/agent-framework/tutorials/agents/middleware?pivots=programming-language-python

7. Memory - https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-memory?pivots=programming-language-python
    The tutorial seems more advanced

8. Observability - https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-observability?pivots=programming-language-python

Other topics TBD

All agents are derived from a common base class, `AIAgent`, which provides a consistent interface for all agent types. 

Agent Framework supports many different types of agents. This tutorial shows you how to create and run an agent with Agent Framework based on the Azure OpenAI Chat Completion service, but all other agent types are run in the same way.

These agents support a wide range of functionality out of the box:

1. Function calling
2. Multi-turn conversations with local chat history management or service provided chat history management
3. Custom service provided tools (e.g. MCP, Code Execution)
4. Structured output
5. Streaming responses


For more information on other agent types and how to construct them, see the [Agent Framework Agent Types](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/?pivots=programming-language-python)

### Our first Agent

Now, let's build an single agent to get started. First, create a chat client for communicating with Azure OpenAI:

```
AZURE_OPENAI_ENDPOINT - The endpoint it should talk to by default
AZURE_OPENAI_API_KEY - The API Key it should use
AZURE_OPENAI_API_VERSION - Inference API version it should use per default
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME - Model deployment name it should use per default
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME - Embedding deployment name it should use per default
```

In [1]:
import os
from dotenv import load_dotenv
from agent_framework.azure import AzureOpenAIChatClient

# Load environment variables
load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Create the client using API key
client = AzureOpenAIChatClient(
    endpoint=endpoint,
    api_key=api_key,
    api_version=api_version,
    deployment_name=deployment
)

Then, create the agent, providing instructions and a name for the agent. To run the agent, call the `.run()` method on the agent instance, providing the user input. 

The agent will return a response object, and accessing the .text property provides the text result from the agent.

Please note that we do not have an chat history, so every message we send to the agent is treated as a new conversation.

In [2]:
simple_agent = client.create_agent(
    instructions="You are an AI assistant that helps users with their questions.",
    name="AI Assistant"
)

### Running the agent 
To run the agent, call the `run` method on the agent instance, providing the user input. The agent will return a response object, and accessing the `.text` property provides the text result from the agent.

To run the agent with streaming, call the `run_stream` method on the agent instance, providing the user input. The agent will stream a list of update objects, and accessing the `.text` property on each update object provides the part of the text result contained in that update.

Python agents support passing keyword arguments to customize each run. The specific options available depend on the agent type, but `ChatAgent` supports many chat client parameters that can be passed to both `run` and `run_stream` methods. Common options include `max_tokens`, `temperature`, `model`, `tools`, `response_format`.

In [3]:
# Regular run
response = await simple_agent.run("What's the capital of Latvia?")
print("Full response:", response.text)

# Streaming run
print("\nStreaming response:")
async for chunk in simple_agent.run_stream("Explain why having a capital for a country is important in two short sentences.", 
                                           max_tokens=30):
    # Each chunk is a partial piece of the model's output
    if chunk.text:
            print(chunk.text, end="", flush=True)
print()

Full response: The capital of Latvia is Riga.

Streaming response:
Having a capital is important as it serves as the central location for a country's government and administration, facilitating effective governance and decision-making. Additionally, a capital


### Message Types and diferent content input examples
simple overview so users know how to work and control those

ChatMessage Object, Images


### Enabling Multi-Turn Conversations and Threading

So far, our agents are stateless and do not maintain any state internally between calls. If you ask them a follow up question, they would have no reference to the prior conversation.

To have a multi-turn conversation with an agent, you need to create an object to hold the conversation state and pass this object to the agent when running it.

An `AgentThread` maintains the conversation state and message history for an agent interaction. It can either use a service-managed thread (via service_thread_id) or a local message store (via message_store), but not both.

To create the conversation state object, call the `get_new_thread()` method on the agent instance.

You can then pass this thread object to the `run` or `run_stream` methods on the agent instance, along with the user input.


In [4]:
agent = client.create_agent(
    instructions="You are an AI assistant that helps users with their questions.",
    name="AI Assistant"
)

# create a thread on the agent instance:
thread = agent.get_new_thread()

user_messages = [
    "Hello!",
    "Which country has Paris as the capital?",
    "What are its neighbouring countries? Give a short one-liner list, please."
]

# Loop through user messages and maintain context
for user_message in user_messages:
    print("*** User:", user_message)
    
    # get response from the agent, passing the same thread
    response = await agent.run(user_message, thread=thread)
    print("*** Agent:", response.text)

print("-" * 25)

# Print the final conversation history from the thread
messages = await thread.message_store.list_messages()
for msg in messages:
    print(f"[{msg.role}] {msg.text}")


*** User: Hello!
*** Agent: Hello! How can I assist you today?
*** User: Which country has Paris as the capital?
*** Agent: Paris is the capital of France. If you have any more questions about France or anything else, feel free to ask!
*** User: What are its neighbouring countries? Give a short one-liner list, please.
*** Agent: France shares its borders with the following countries:

1. Belgium
2. Luxembourg
3. Germany
4. Switzerland
5. Italy
6. Monaco
7. Spain
8. Andorra
9. Brazil (via Guiana)
10. Suriname (via Guiana)

If you need more information about any of these countries, just let me know!
-------------------------
[user] Hello!
[assistant] Hello! How can I assist you today?
[user] Which country has Paris as the capital?
[assistant] Paris is the capital of France. If you have any more questions about France or anything else, feel free to ask!
[user] What are its neighbouring countries? Give a short one-liner list, please.
[assistant] France shares its borders with the following

### Single agent with multiple conversations

It is possible to have multiple, independent conversations with the same agent instance, by creating multiple `AgentThread` objects. These threads can then be used to maintain separate conversation states for each conversation. The conversations will be fully independent of each other, since the agent does not maintain any state internally.


In [5]:
# Create two threads for two separate conversations
thread1 = agent.get_new_thread()
thread2 = agent.get_new_thread()

# First messages for each thread
result1 = await agent.run("Suggest 1 key attraction for Paris trip. Keep it brief.", thread=thread1)
print("Thread 1:", result1.text)

result2 = await agent.run("Suggest 1 key attraction for Tokyo trip. Keep it brief.", thread=thread2)
print("Thread 2:", result2.text)

# Continue each conversation independently
result3 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread1
)
print("Thread 1:", result3.text)

result4 = await agent.run(
    "Now suggest one morning activity. One short sentence.",
    thread=thread2
)
print("Thread 2:", result4.text)

Thread 1: Visit the Eiffel Tower for stunning views of the city and a quintessential Parisian experience.
Thread 2: Visit the Senso-ji Temple in Asakusa, Tokyo’s oldest temple, known for its stunning architecture and lively Nakamise shopping street leading up to it.
Thread 1: Stroll through the charming streets of Montmartre and enjoy a coffee at a local café while admiring the artistic vibe.
Thread 2: Take a peaceful stroll through the tranquil Shinjuku Gyoen National Garden, surrounded by beautiful landscapes and seasonal blooms.


### Agent Tools
Tooling support may vary considerably between different agent types. Some agents may allow developers to customize the agent at construction time by providing external function tools or by choosing to activate specific built-in tools that are supported by the agent. 

The `ChatAgent` is an agent class that can be used to build agentic capabilities on top of any inference service. It comes with support for:

1. Using your own function tools with the agent
2. Using built-in tools that the underlying service may support
3. Using hosted tools like web search and MCP (Model Context Protocol) servers

Let's try to provide function tools during agent construction:

In [6]:
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from datetime import datetime

def get_time() -> str:
    return datetime.now().strftime("%H:%M:%S")

# Sample function tool
def get_weather(
    location: Annotated[str, "The location to get the weather for."],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

# Agent with agent-level tool available  for all runs
agent = ChatAgent(
    chat_client=AzureOpenAIChatClient(),
    instructions="You are a helpful assistant",
    tools=[get_time] 
)

# This run has access to both get_time (agent-level) and get_weather (run-level)
result = await agent.run(
    "What's the weather and time in New York?",
    tools=[get_weather]  # Additional tool for this run
)

print(result.text)

The current time in New York is 16:34:48, and the weather is cloudy with a high of 15°C.


In [ ]:
# You can also create a class that contains multiple function tools as methods. 
from typing import Annotated, List, Tuple
from datetime import datetime


class NotesTools:
    def __init__(self):
        self.notes = []  # Store notes as (timestamp, note)

    def list_notes(self) -> Annotated[List[Tuple[str, str]], "Returns all notes as (timestamp, note)."]:
        """Return all notes with their timestamps."""
        if not self.notes:
            return "No notes available."
        return self.notes

    def write_note(
        self,
        note: Annotated[str, "The note message to save."],
    ) -> str:
        """Save a note with the current timestamp."""
        timestamp = datetime.now().isoformat()
        self.notes.append((timestamp, note))
        return f"Note added at {timestamp}."

    
# create tools instance
tools = NotesTools()

# create agent with tools
agent = client.create_agent(
    instructions="You are a helpful assistant that can take notes and show them.",
    tools=[tools.write_note, tools.list_notes]
)

# create a thread for multi-turn conversation
thread = agent.get_new_thread()

user_messages = [
    "Find new soup recipes.",
    "Find new project opportunities at university.",
    "Don't forget to call Jess to say happy birthday.",
    "Can you list all my notes?"
]

for m in user_messages:
    print("*** User:", m)
    response = await agent.run(m, thread=thread)
    print("*** Agent:", response.text)

*** User: Hello!
*** Agent: Hello! How can I assist you today?
*** User: Please write a note saying 'Meeting with client at 3 PM'.
*** Agent: I've written the note: "Meeting with client at 3 PM". If you need anything else, just let me know!
*** User: Add another note: 'Prepare slides for tomorrow'.
*** Agent: I've added the note: "Prepare slides for tomorrow." Let me know if you need anything else!
*** User: Can you list all my notes?
*** Agent: Here are all your notes:

1. **Meeting with client at 3 PM** - Timestamp: 2025-10-25 16:34:59
2. **Prepare slides for tomorrow** - Timestamp: 2025-10-25 16:35:00

If you need anything else, feel free to ask!


### TBD - Structured Output with Agents
Sometimes it might be helpful to have our agent return its response in a fixed format. For this we can use the Response Format feature. This is similar to OpenAI's Structured Output feature and forces the model's output into a pre-defined structure:


## Exercise - implement an agent with function tools

In this exercise, you'll create an agent that manages tasks with priorities and deadlines. This will help you understand how to create and expose function tools to an agent and run a multi-turn conversation.

#### Task:
1. Create a `TaskManagerTools` class with the following functions:
    - `add_task(name, priority, deadline)` - Adds a new task with a description, priority (high, medium, low), and deadline in DD-MM format.
    - `list_tasks()` - Returns all tasks with their details.
    - `filter_by_priority(priority)` - Returns tasks that match the given priority.

2. Annotate parameters with `Annotated` to provide descriptions for better LLM understanding.
3. Create an agent with clear instructions and register function tools
4. Test the agent with a multi-turn conversation

In [8]:
# will be hidden in HTML, here for test/review
from typing import Annotated, List, Dict
from datetime import datetime

class TaskManagerTools:
    def __init__(self):
        self.tasks: List[Dict[str, str]] = []  # expected list of dicts, each task: {"name": str, "priority": str, "deadline": str}

    def add_task(
        self,
        name: Annotated[str, "The task description."],
        priority: Annotated[str, "Priority level: high, medium, low."],
        deadline: Annotated[str, "Deadline in format like 'September 30' or '30.10'."],
    ) -> str:
        """Add a new task with priority and deadline (stored as DD-MM)."""
        deadline_str = deadline  # Default to raw input
        try:
            # Try parsing "September 30" or "30.10"
            if "." in deadline:  # Format like 30.10
                parsed_date = datetime.strptime(deadline, "%d-%m")
            else:  # Format like September 30
                parsed_date = datetime.strptime(deadline, "%B %d")
            deadline_str = parsed_date.strftime("%d-%m")
        except ValueError:
            # If parsing fails, keep raw input
            pass

        self.tasks.append({"name": name, "priority": priority, "deadline": deadline_str})
        return f"Task '{name}' added with priority {priority} and deadline {deadline_str}."

    def list_tasks(self) -> Annotated[List[Dict[str, str]], "Returns all tasks with details."]:
        """List all tasks."""
        return self.tasks if self.tasks else "No tasks available."

    def filter_by_priority(
        self,
        priority: Annotated[str, "Priority level to filter: high, medium, low."],
    ) -> List[Dict[str, str]]:
        """Return tasks matching the given priority."""
        filtered = [task for task in self.tasks if task["priority"] == priority]
        return filtered if filtered else f"No tasks with priority {priority}."


# Create agent with tools
tools = TaskManagerTools()

agent = client.create_agent(
    instructions="You are a helpful and precise assistant that manages tasks with priorities and deadlines.",
    tools=[tools.add_task, tools.list_tasks, tools.filter_by_priority]
)

# Multi-turn conversation
thread = agent.get_new_thread()
user_messages = [
    "Add a task to prepare workshop slides, high priority by 30.10",
    "Send invites by december 30th",
    "I need to send an email to Kate by nov 4",
    "List all tasks.",
    "Show me tasks with low priority."
]

for m in user_messages:
    print("*** User:", m)
    response = await agent.run(m, thread=thread)
    print("*** Agent:", response.text)


*** User: Add a task to prepare workshop slides, high priority by 30.10
*** Agent: The task to prepare workshop slides has been added with high priority and a deadline of 30.10.
*** User: Send invites by december 30th
*** Agent: The task to send invites has been added with medium priority and a deadline of December 30th.
*** User: I need to send an email to Kate by nov 4
*** Agent: The task to send an email to Kate has been added with medium priority and a deadline of November 4th.
*** User: List all tasks.
*** Agent: Here are all your tasks:

1. **Prepare workshop slides**
   - Priority: High
   - Deadline: 30.10

2. **Send invites**
   - Priority: Medium
   - Deadline: 30.12

3. **Send email to Kate**
   - Priority: Medium
   - Deadline: 04.11
*** User: Show me tasks with low priority.
*** Agent: There are currently no tasks with low priority.


### Agent Middleware

Middleware in the Agent Framework provides a powerful way to intercept, modify, and enhance agent interactions at various stages of execution. You can use middleware to implement cross-cutting concerns such as logging, security validation, error handling, and result transformation without modifying your core agent or function logic.

Function-based middleware is the simplest way to implement middleware using async functions. This approach is ideal for stateless operations and provides a lightweight solution for common middleware scenarios.

1. Agent Middleware
2. Function Middleware
3. Chat Middleware


In [9]:
from datetime import datetime
from agent_framework import FunctionInvocationContext
from typing import Callable, Awaitable
from agent_framework import agent_middleware

def get_time():
    """Get the current time."""
    return datetime.now().strftime("%H:%M:%S")

# middleware hooks onto function invocation events
async def logging_function_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """Middleware that logs function execution."""
    print(f"[Function] Calling {context.function.name}")

    await next(context) # executes the actual function or any next MW

    print(f"[Function] {context.function.name} completed")
    print(f"[Function] Result: {context.result}")


# agent-level MW is applied to the entire agent run
@agent_middleware  # Explicitly marks as agent middleware
async def logging_agent_middleware(context, next):    # don't need to explicitly declare type annotations
    """Agent middleware with decorator - types are inferred."""
    print(f"[Agent] BEFORE execution")
    await next(context)     # executes the agent logic
    print(f"[Agent] AFTER execution | Agent output: {context.result}")

agent = client.create_agent(
    name="TimeAgent",
    instructions="You can tell the current time.",
    tools=[get_time],    
    middleware=[logging_function_middleware, logging_agent_middleware]
)

result = await agent.run("What time is it?")
print("Final response: ", result.text)

print("----------------------------------")

# the following response doesn't invoke get_time() so logging_function MW won't fire
result = await agent.run(
    "Can you help?"
)

print("Final response: ", result.text)

[Agent] BEFORE execution
[Function] Calling get_time
[Function] get_time completed
[Function] Result: 16:38:38
[Agent] AFTER execution | Agent output: The current time is 16:38:38.
Final response:  The current time is 16:38:38.
----------------------------------
[Agent] BEFORE execution
[Agent] AFTER execution | Agent output: Of course! What do you need help with?
Final response:  Of course! What do you need help with?


### Example from SK where logging was added

In [17]:
from typing import TypedDict, Annotated, List, Optional

class LightModel(TypedDict):
    id: int
    name: str
    is_on: bool | None
    brightness: int | None
    hex: str | None

class LightsTools:
    def __init__(self, lights: list[LightModel]): # stores a list of lights in self.lights
        self.lights = lights

    def get_lights(self) -> List[LightModel]:                
        """Gets a list of lights and their current state."""
        return self.lights

    def get_state(
        self, id: Annotated[int, "The ID of the light"] #  Annotated adds documentation for LLMs
    ) -> Optional[LightModel]:
        """Gets the state of a particular light."""
        for light in self.lights:
            if light["id"] == id:
                return light
        return None
    
    def change_state(
        self, id: Annotated[int, "The ID of the light"], new_state: LightModel
    ) -> dict:
        """Changes the state of the light and returns previous/current info."""
        for light in self.lights:
            if light["id"] == id:
                previous = light.copy()  # snapshot before change
                light["is_on"] = new_state.get("is_on", light["is_on"])
                light["brightness"] = new_state.get("brightness", light["brightness"])
                light["hex"] = new_state.get("hex", light["hex"])
                return {
                    "previous": previous,
                    "current": light
                }
        return None



In [23]:
lights = [
    {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
    {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
    {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
]

# data to instantiate a class and pass methods as tools
tools = LightsTools(lights=lights)

In [27]:

agent = client.create_agent(
    instructions="""
    You control and manage smart lights.
    - Check previous states to check if an action needs to be taken
    - When turning lights on or off, do NOT change brightness or hex unless the user explicitly asks.
    - When changing brightness or hex, only modify those values if requested.""",
    tools=[tools.get_lights, tools.get_state, tools.change_state]
)

result = await agent.run(
    "turn on all lamps",
    )

print(result)
print("Final data:", tools.lights)

All lamps are already turned on, so no action is required.
Final data: [{'id': 1, 'name': 'Table Lamp', 'is_on': True, 'brightness': 100, 'hex': 'FF0000'}, {'id': 2, 'name': 'Porch light', 'is_on': True, 'brightness': 50, 'hex': '00FF00'}, {'id': 3, 'name': 'Chandelier', 'is_on': True, 'brightness': 75, 'hex': '0000FF'}]


In the following exercise, we'll be adding middleware for easier debugging and understanding of agent behavior, because currently we can't see anything besides agent's response.
We'll add function middleware which intercepts function calls within agents, chat middleware dor OpenAI usage logging and agent-level middleware which is persistent across all runs for tool call counting

In [28]:
import logging
import time
from agent_framework import function_middleware, agent_middleware, chat_middleware

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s", force=True
)
logger = logging.getLogger(__name__)

@agent_middleware       # decorator allows to skip type annotations like AgentRunContext
async def agent_middleware(context,next                           
) -> None:
    start = time.perf_counter()

    logger.info(f"[Agent] Starting execution")

    await next(context)

    # Calculate total duration
    duration = time.perf_counter() - start
    logger.info(f"[Agent] Agent run completed in {duration:.4f}s")

@chat_middleware 
async def chat_middleware(context, next
) -> None:
    logger.info(f"[Chat] Sending {len(context.messages)} messages to AI")

    await next(context)

    logger.info("[Chat] Response received")

@function_middleware        
async def function_middleware(context, next
) -> None:
    logger.info(f"[Function] Calling - {context.function.name} | Args: {context.arguments}")
    start = time.perf_counter()

    await next(context)

    duration = time.perf_counter() - start
    logger.info(f"[Function] Completed \"{context.function.name}\" in {duration:.6f}s | Result: {context.result}")


agent = client.create_agent(
    instructions="""
    You control and manage smart lights.
    - Check previous states to check if an action needs to be taken
    - When turning lights on or off, do NOT change brightness or hex unless the user explicitly asks.
    - When changing brightness or hex, only modify those values if requested.""",
    tools=[tools.get_lights, tools.get_state, tools.change_state],
    middleware=[agent_middleware, chat_middleware, function_middleware]
)

result = await agent.run(
    "turn off all lights and give me their final state"
    )

print("Assistant >" + str(result))

2025-10-25 17:01:30,846 - INFO - [Agent] Starting execution
2025-10-25 17:01:30,846 - INFO - [Chat] Sending 2 messages to AI
2025-10-25 17:01:31,757 - INFO - HTTP Request: POST https://semantic-kernel1.openai.azure.com/openai/deployments/sk-gpt-4o-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-10-25 17:01:31,767 - INFO - [Chat] Response received
2025-10-25 17:01:31,770 - INFO - [Function] Calling - get_lights | Args: 
2025-10-25 17:01:31,772 - INFO - Function name: get_lights
2025-10-25 17:01:31,772 - INFO - Function get_lights succeeded.
2025-10-25 17:01:31,781 - INFO - [Function] Completed "get_lights" in 0.006948s | Result: [{'id': 1, 'name': 'Table Lamp', 'is_on': True, 'brightness': 100, 'hex': 'FF0000'}, {'id': 2, 'name': 'Porch light', 'is_on': True, 'brightness': 50, 'hex': '00FF00'}, {'id': 3, 'name': 'Chandelier', 'is_on': True, 'brightness': 75, 'hex': '0000FF'}]
2025-10-25 17:01:31,784 - INFO - [Chat] Sending 4 messages to AI
2025-10-25 17:01:33

Assistant >All lights have been turned off. Here are their final states:

1. **Table Lamp**
   - Previous State: On
   - Current State: Off
   - Brightness: 100
   - Hex: FF0000

2. **Porch Light**
   - Previous State: On
   - Current State: Off
   - Brightness: 50
   - Hex: 00FF00

3. **Chandelier**
   - Previous State: On
   - Current State: Off
   - Brightness: 75
   - Hex: 0000FF


### TO-DO Exercise adding middleware to agents 
Based on SK's 1.1 "Creating a Content Filter" 

### TO-DO Exercise Create Your Own Function Tools
Based on SK's 2.1 "Create Your Own Plugin"